In [ ]:
import pathlib
import pandas as pd
import pyautogui as pyg
import time
import smtplib
import os
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders

In [ ]:
email = pd.read_excel(r"C:\Users\braio\OneDrive\Área de Trabalho\Github\python\Projeto Automação de Processos\Projeto AutomacaoIndicadores\Bases de Dados\Emails.xlsx")
lojas = pd.read_csv(r"C:\Users\braio\OneDrive\Área de Trabalho\Github\python\Projeto Automação de Processos\Projeto AutomacaoIndicadores\Bases de Dados\Lojas.csv", encoding="latin1", sep=";")
vendas = pd.read_excel(r"C:\Users\braio\OneDrive\Área de Trabalho\Github\python\Projeto Automação de Processos\Projeto AutomacaoIndicadores\Bases de Dados\Vendas.xlsx")

In [ ]:
vendas = vendas.merge(lojas, on='ID Loja')

In [ ]:
dicionario_loja = {}
for loja in lojas['Loja']:
    dicionario_loja[loja] = vendas.loc[vendas['Loja'] == loja, :]

In [ ]:
dia_indicador = vendas['Data'].max()

In [ ]:
#identificar se a pasta existe
caminho_backup = pathlib.Path(r"C:\Users\braio\OneDrive\Área de Trabalho\Github\python\Projeto Automação de Processos\Projeto AutomacaoIndicadores\Backup Arquivos Lojas")

arquivo_pasta_backup = caminho_backup.iterdir()
lista_nome_backup = [arquivo.name for arquivo in arquivo_pasta_backup]

for loja in dicionario_loja:
    if loja not in lista_nome_backup:
        nova_pasta = caminho_backup / loja
        nova_pasta.mkdir()

#salvar dentro da pasta
    nome_arquivo = f'{dia_indicador.month}_{dia_indicador.day}_{loja}.xlsx'
    local_arquivo = caminho_backup / loja / nome_arquivo

    dicionario_loja[loja].to_excel(local_arquivo)


In [ ]:
loja = 'Norte Shopping'
venda_loja = dicionario_loja[loja]
venda_loja_dia = venda_loja.loc[venda_loja['Data'] == dia_indicador, :]

#faturamento
faturamento_ano = venda_loja['Valor Final'].sum()
faturamento_dia = venda_loja_dia['Valor Final'].sum()

#diversidade de produtos
qtde_produto_ano = len(venda_loja['Produto'].unique())
qtde_produto_dia = len(venda_loja_dia['Produto'].unique())

#ticket médio
venda_loja = venda_loja.drop('Data', axis=1)
valor_venda = venda_loja.groupby('Código Venda').sum()
ticket_medio_ano = valor_venda['Valor Final'].mean()

venda_loja_dia = venda_loja_dia.drop('Data', axis=1)
valor_venda_dia = venda_loja_dia.groupby('Código Venda').sum()
ticket_medio_dia = valor_venda_dia['Valor Final'].mean()

In [ ]:
#definição de metas

meta_faturamento_dia = 1000
meta_faturamento_ano = 1650000
meta_qtdeprodutos_dia = 4
meta_qtdeprodutos_ano = 120
meta_ticketmedio_dia = 500
meta_ticketmedio_ano = 500

Acessar o Gmail

In [ ]:
smtp_server = 'smtp.gmail.com'
smtp_port = 587
smtp_username = remetente = ''
smtp_password = ''

for loja in lojas['Loja']:
    destinatario = email.loc[email['Loja'] == loja, 'E-mail'].values[0]
    assunto = f"OnePage Dia {dia_indicador.day}/{dia_indicador.month} - Loja {loja}"
    gerente = email.loc[email['Loja'] == loja, 'Gerente'].values[0]
    arquivo = caminho_backup / loja / f"{dia_indicador.month}_{dia_indicador.day}_{loja}.xlsx"         
    


    mensagem = MIMEMultipart()
    mensagem['From'] = remetente
    mensagem['To'] = destinatario
    mensagem['Subject'] = assunto


    if faturamento_dia >= meta_faturamento_dia:
        cor_fat_dia = 'green'
    else:
        cor_fat_dia = 'red'
    if faturamento_ano >= meta_faturamento_ano:
        cor_fat_ano = 'green'
    else:
        cor_fat_ano = 'red'
    if qtde_produto_dia >= meta_qtdeprodutos_dia:
        cor_qtde_dia = 'green'
    else:
        cor_qtde_dia = 'red'
    if qtde_produto_ano >= meta_qtdeprodutos_ano:
        cor_qtde_ano = 'green'
    else:
        cor_qtde_ano = 'red'
    if ticket_medio_dia >= meta_ticketmedio_dia:
        cor_ticket_dia = 'green'
    else:
        cor_ticket_dia = 'red'
    if ticket_medio_ano >= meta_ticketmedio_ano:
        cor_ticket_ano = 'green'
    else:
        cor_ticket_ano = 'red'

    corpo = f'''
    <p>Bom dia, {gerente}!</p>
    
    <p>O resultado de ontem <strong>({dia_indicador.day}/{dia_indicador.month})</strong> da <strong>Loja {loja}</strong> foi:</p>

    <div style="border: 1px solid black; max-width: 50%;">
        <table>
            <tr style="text-indent: 17px;">
                <th>Indicador</th>
                <th>Valor dia</th>
                <th>Meta dia</th>
                <th>Cenário dia</th>
            </tr>
            <tr>
                <td>Faturamento</td>
                <td style="text-align: center">{faturamento_dia}</td>
                <td style="text-align: center">{meta_faturamento_dia}</td>
                <td style="text-align: center"><font color="{cor_fat_dia}">◙</font></td>
            </tr>
            <tr>
                <td>Diversidade de produtos</td>
                <td style="text-align: center">{qtde_produto_dia}</td>
                <td style="text-align: center">{meta_qtdeprodutos_dia}</td>
                <td style="text-align: center"><font color="{cor_qtde_dia}">◙</font></td>
            </tr>
            <tr>
                <td>Ticket médio</td>
                <td style="text-align: center">{ticket_medio_dia}</td>
                <td style="text-align: center">{meta_ticketmedio_dia}</td>
                <td style="text-align: center"><font color="{cor_ticket_dia}">◙</font></td>
            </tr>
        </table>
        <br>
        <br>
        <table>
            <tr style="text-indent: 17px;">
                <th>Indicador</th>
                <th>Valor dia</th>
                <th>Meta dia</th>
                <th>Cenário dia</th>
            </tr>
            <tr>
                <td>Faturamento</td>
                <td style="text-align: center">{faturamento_ano}</td>
                <td style="text-align: center">{meta_faturamento_ano}</td>
                <td style="text-align: center"><font color="{cor_fat_ano}">◙</font></td>
            </tr>
            <tr>
                <td>Diversidade de produtos</td>
                <td style="text-align: center">{qtde_produto_ano}</td>
                <td style="text-align: center">{meta_qtdeprodutos_ano}</td>
                <td style="text-align: center"><font color="{cor_qtde_ano}">◙</font></td>
            </tr>
            <tr>
                <td>Ticket médio</td>
                <td style="text-align: center">{ticket_medio_ano}</td>
                <td style="text-align: center">{meta_ticketmedio_ano}</td>
                <td style="text-align: center"><font color="{cor_ticket_ano}">◙</font></td>
            </tr>
        </table>
    </div>
    
    <p>Segue em anexo a planilha com todos os dados para mais detalhes.</p>

    <p>Qualquer duvida estou a disposição.</p>
    <p>Atenciosamente, Yohan.</p>
    '''
    mensagem.attach(MIMEText(corpo, "html"))


    if os.path.exists(arquivo):
        with open(arquivo, "rb") as arquivo_aberto:
            anexo = MIMEBase("application", "octet-stream")
            anexo.set_payload(arquivo_aberto.read())
            encoders.encode_base64(anexo)
            anexo.add_header("Content-Disposition", f"attachment; filename={os.path.basename(arquivo)}")
            mensagem.attach(anexo)
    else:
        print("Erro: O arquivo não foi encontrado!")

    try:
        servidor_smtp = smtplib.SMTP(smtp_server, smtp_port)
        servidor_smtp.starttls()  # Inicia a comunicação criptografada
        servidor_smtp.login(smtp_username, smtp_password)
        servidor_smtp.sendmail(remetente, destinatario, mensagem.as_string())
        print('E-mail enviado com sucesso!')
    except Exception as e:
        print(f'Erro ao enviar e-mail: {e}')
    finally:
        servidor_smtp.quit()